# **Logbook MuLife**

### **I° parte : Caratterizzazione del telescopio/bersaglio**

Abbiamo scelto la seguente configurazione : 
- PMT07, PMT04 e PMT02 come telescopio;
- PMT08-09-10-11 come bersaglio;
- PMT01 come PMT di VETO;

|PMT|Lunghezza[cm]|Larghezza[cm]|Spessore[cm]|Errore[cm]|
|:-:|:-----------:|:-----------:|:----------:|:--------:|
|07, 04, 02, 01|49.8|27.5|2.3|0.1|
|08 + 09 + 10 + 11|30.0|31.0|31.0|0.1|

Fra lo 02 e 04 ho 37 cm di distanza, fra 07 e 04 ne ho 27 cm.

Iniziamo con la scelta del punto di lavoro.

Alimentiamo inizialmente il PMT07 a 1750 V , il PMT04 a 1720 V e il PMT02 a 1750 V. Le rivedremo in seguito.
Abbiamo poi usato la configurazione dell'esperienza preliminare (discriminatore + unità di coincidenza + contatore) per caratterizzare il telescopio misurandone l'efficenza. Ricostruiamo l'efficenza in funzione della tensione. 

Riporto in seguito il punto di lavoro scelto per i vari PMT : 

|PMT|$HV$[kV]|$\varepsilon$[%]|$V_{soglia}$[mV]|$w_{dsr}$[ns]|$D_{fake}$|
|:-:|:-:|:----------:|:----------:|:-----:|:------:|
|07|1.750|79|-30|50|0.64|
|04|1.700||||
|02|1.750|88|-30|50|0.65|
|OR|1.250|76|-50|50|0.095|
|01|1.720|29|-30|100|(vedi giù)|

($w_{dsr}$ è  la durata del segnale in uscita dal discriminatore). 

Il PMT07 ha un'accettanza di 0.5, mentre il 02 di 0.4 (già incluso nell'efficenza riportata in tabella). E' stata determinata col Montecarlo che ho su Ubuntu. Il resto dei cacloli li abbiamo eseguiti con lo script EffCurves.py.

Va sottolineato che tale setup è stato scelto provissoriamente per eseguire delle misure di test.



### **II° parte : Logica d'acquisizione**

Serve una logica che ci dica quando un muone viene fermato (START) e decade (STOP). Il segnale di START sarà identificato da una coincidenza fra :

- PMT07, PMT04 e PMT02 accesi;

- OR di PMT09, PMT10, PMT11, PMT12 acceso;

- nessun segnale in PMT01 (VETO);

Introduciamo una finestra di 100 ns col dual-timer ond'evitare doppi start ravvicinati, per poi rinviare il segnale del timer a una seconda unità di timing per riavere un impulso stretto che faccia un nuovo segnale di start. Il ritardo fra lo START originale e quello in uscita dall'unità di timer introdotto è di circa 50 ns. 

Come segnale di STOP scegliamo una coincidenza fra :
- OR di bersaglio; 
- PMT01 (corrisponde all'elettrone prodotto che muore nel bersaglio o al più passa sotto);
- segnale di 20 $\mu s$ generato dal timer sullo START (evita di prendere segnali troppo lontani);
- segnale da 100 ns sugli START (evita STOP troppo vicini allo START).

Il segnale di VETO è preso con una linea di trasmissione da 1 ns di ritardo, in modo che sia anticipato il più possibile rispetto agli altri segnali. 

Quello del telescopio è raccolto con cavi da 5 ns, sempre per far in modo che il ritardo sia sufficente.

Il ritardo introdotto dall' unità di OR è 6 ns, quindi prendiamo il segnale dal bersaglio con un cavo da 4 ns.

Metto una rappresentazione schematica della logica:

![alt text](Grafici/LogicaMuLife.png)

Per ora leviamo il VETO, poi vediamo se rimetterlo.

### **III° parte : Acquisizione e analisi dati**

Sistemiamo il convertitore NIM-TTL, per poi inviare il segnale a DEO10-nano. 
I file sono sempre salvati come FIFOread(data). Il file di output è organizzato in questo modo:

- la prima colonna indica il canale scattato come potenza di 2 (START $\rightarrow$ 1/ STOP $\rightarrow$ 2)
- la seconda colonna indica il numero di periodi di clock scattati ($T_{clk}$ = 5 ns) 

Abbiamo relativamente pochi STOP, quindi abbiamo bisogno di take molto lunghi.

Per calibrare il programma gli mandiamo un'onda quadra di frequenza nota. Per generarla usiamo un'unità di timer:

- Uso EndMarker del primo timer per generare un impulso e lo mando al secondo timer in START;
- Uso EndMarker del secondo timer per mandarlo poi allo START al primo;

L'onda ha frequenza 1.071 Hz. Dopo averla convertita da NIM a TTL la diamo in pasto al programma di acquisizione FIFO.

Abbiamo inoltre deciso di calcolare il ritardo fra i due canali. Per farlo abbiamo mandato la stessa onda quadra su due canali diversi.

La costante di calibrazione dopo aver analizzato un dataset di 3 minuti viene $a = 4.98892 \pm 0.00009 $ ns/dg .

Il ritardo medio è di al più un digit (in tempo è parente della scala di sopra).


### **IV° parte : Ottimizzazione dell'apparato**

In primo luogo vogliamo rivedere la curva di efficenza del PMT01.

Lo 01 è problematico, abbiamo grossi problemi di accettanza usando il bersaglio completo e il PMT02, l'efficenza è attorno al 29%. Per ovviare al problema abbiamo rimisurato la curva di efficenza con il PMT07 e l'OR di PMT09 e PMT11. 
In questo modo abbiamo più vincolo geometrico (angolo solido più stretto) e pertanto ci aspettiamo una misura più affidabile.

Misurandola in questo modo viene attorno al 60% a 1700 V, con un numero di fake pari a 0.095.

In seguito abbiamo analizzato i PMT del blocco in modo da ottimizzarli singolarmente.
In particolare: 
- Per il PMT09 abbiamo usato 02-09-11;
- Per il PMT11 abbiamo usato 09-11-01;
- Per il PMT08 abbiamo usato 02-08-10;
- Per il PMT10 abbiamo usato 07-08-10 (pensato in modo da stringere l'angolo solido in quanto il PMT01 è fuori asse con i PMT più interni del blocco);

Riporto una tabella:

|PMT|$HV$[kV]|$\varepsilon$[%]|$D_{fake}$|
|:-:|:-:|:----------:|:------:|
|08|1.26|78|0.067|
|09|1.32|85|0.079|
|10|1.26|58|0.054|
|11|1.26|87|0.023|

(Soglie e durate dei segnali discriminati sono rimasti invariati, qui ho riportato i valori più vicini a quelli che abbiamo sulla curva, dopo correggi la curva con le tensioni in modo truffaldino in sede di relazione)

Abbiamo riallungato la durata del VETO a 150 ns per evitare i falsi START causati dal rimbalzo del segnale  di OR.

Abbiamo tolto in seguito le finestre di STOP dei 20 $\mu s$ e 100 ns, in quanto constatato che non era rilevante in sede di presa dati e che è più semplice tenerne conto in sede di analisi dati. 

Dopo aver mappato tutti i cavi abbiamo fatto delle misure di ritardo, in particolare fra start ripulito e creazione della finestra di 100 ns ho 10 ns di ritardo.

Inoltre abbiamo ripulito il segnale di STOP con un metodo simile al segnale di START 150 ns. Considerate le durate dei segnali, questo significa che dopo lo STOP "forzato" dallo START ho 75 ns dopo il quale non posso avere nuovi STOP (vengono tutti inglobati nel primo).
Cerchiamo di far arrivare il segnale START e STOP forzato simultaneamente a FIFO.

Abbiamo ripristinato la finestra perché se no prendiamo troppi OR. La finestra è sempre di 20 $\mu s$, la sovrapposizione con gli STOP è di circa di 10 ns. Abbiamo anche rinunciato a ripulire lo STOP con il timer, ravisando che faceva perdere delle coincidenze. Non è vero lo teniamo, siamo un po' confusi in sti giorni.

Non torna, proviamo a mettere ciascun PMT del blocco in coincidenza col GATE e proviamo a correlare i falsi STOP con gli STOP fisici. Le corrispondenze fra canali e PMT è : 

- 0-START
- 1-STOP
- 2-8&GATE
- 3-9&GATE
- 4-10&GATE
- 5-11&GATE


### **La mia su $N_+/N_-$**

Ragiono così : 
- La probabilità di trovare un muone con carica $\pm e$ decade esponenzialmente:

$$
\begin{equation}
P_{\pm} = \frac{1}{\tau_{\pm}}e^{-t/\tau_{\pm}}
\end{equation} 
$$

- Se noi abbiamo un set di misure di tempi quello che vediamo è la sovrapposizione degli istogrammi corrispondenti ai tempi di vita per $\mu^{\pm}$, ossia la somma : 

$$
\begin{equation}
N(t) = \frac{N}{2\tau_{+}}e^{-t/\tau_{+}} + \frac{N}{2\tau_{-}}e^{-t/\tau_{-}} = N_{+}(t) + N_{-}(t)
\end{equation} 
$$

N sono le occorrenze totali. Possiamo accedere alla media e deviazione standard : 

$$
\begin{align}

\mu = \frac{1}{N}\int{t N(t)} = \frac{\tau_{+} + \tau_{-}}{2} \\

<t^{2}> = \frac{1}{N}\int{t^{2} N(t)} = \tau_{+}^2 + \tau_{-}^2 \\

\implies \sigma^2  = <t^{2}> -\mu^2 

\end{align}
$$

Da cui posso esprimere le vite medie partendo dalla distribuzione campionata:

$$
\begin{equation}
\tau_{\pm} = \mu \left(1 \pm \frac{1}{2}\sqrt{1-\left(\frac{\sigma}{\mu}\right)^{2}}\right)
\end{equation}
$$

- Il rapporto $N_{+}/N_{-}$ sarà: 

$$
\begin{equation}
N_{+}/N_{-} = \frac{\tau_{-}}{\tau_{+}} e^{-t \left(\frac{1}{\tau_{+}} + \frac{1}{\tau_{-}}\right)}
\end{equation}
$$

Ci interessa la media temporale, divido per $\mu$, non mi aspetto di avere molte occorrenze oltre il tempo medio: 

$$
\begin{equation}
<N_{+}/N_{-}> = \frac{\tau_{-}}{\tau_{+}} \frac{1}{\mu} \int{e^{-t \left(\frac{1}{\tau_{+}} + \frac{1}{\tau_{-}}\right)}} dt = \frac{2}{\left(1+ \frac{\tau_{+}}{\tau_{-}}\right)^2} 
\implies <N_{+}/N_{-}> = \frac{1}{2}\left(1-\frac{1}{2}\sqrt{1-\left(\frac{\sigma}{\mu}\right)^2}\right)^2
\end{equation}
$$


